<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/cdvae_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab jarvis-tools
import condacolab
condacolab.install()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 52.7 MB/s eta 0:00:00
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


In [ ]:
import os

os.chdir('/content')
# Clone ALIGNN repo to get example folder
if not os.path.exists('CDVAE'):
  os.makedirs('CDVAE')
os.chdir('CDVAE')
if not os.path.exists('cdvae'):
  !git clone https://github.com/knc6/cdvae.git

os.chdir('cdvae')
if not os.path.exists('WABDB'):
  os.makedirs('WABDB')
if not os.path.exists('HYDRA_JOBS'):
  os.makedirs('HYDRA_JOBS')
!conda env create -n cdvae --file env.ci.yml
!conda activate cdvae
!pip install -e .
!export HYDRA_FULL_ERROR=1
!export PROJECT_ROOT=$PWD
!export WABDB=$PWD/WABDB
!export WABDB_DIR=$PWD/WABDB
!export HYDRA_JOBS=$PWD/HYDRA_JOBS
#coverage run -m pytest

Cloning into 'cdvae'...
remote: Enumerating objects: 577, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 577 (delta 7), reused 16 (delta 7), pack-reused 561
Receiving objects: 100% (577/577), 189.20 MiB | 34.61 MiB/s, done.
Resolving deltas: 100% (241/241), done.
Updating files: 100% (103/103), done.